##### Author: [Juste Nyirimana](https://www.linkedin.com/in/juste-nyirimana-25a534144/)

# UmojaHack Rwanda: Expresso Churn Prediction Challenge

When customers leave, this can be very costly for the company, thus firms are interested in predicting churn beforehand. Having that information in hand can help companies change their strategy to retain customers.

In this starter notebook, we'll walk through the competition. We will show you how to load the data and do a quick exploratory analysis. Then, we will train a simple model, make some predictions, and then submit those predictions to the competition.

The data we have describes 2.5 million [Expresso](https://www.expressotelecom.sn/) clients. Expresso is an African telecommunications services company that provides telecommunication services in five African markets: Mauritania, Senegal, Guinea, and Ghana. Expresso offers a wide range of products and services to meet the needs of customers.

The objective of this hackathon is to develop a predictive model that determines the likelihood for a customer to churn - to stop purchasing airtime and data from Expresso. Let's dig in!

## Table of Contents:
&nbsp;&nbsp;1. [LOADING THE DATA](#1)
   
&nbsp;&nbsp;2. [EXPLORING THE DATA](#2)   

&nbsp;&nbsp;3. [BUILDING SOME MODELS](#3)   

&nbsp;&nbsp;4. [GENERATING THE PREDICTIONS FOR THE TEST SET](#4)

# LOADING THE DATA

On the [data description page](https://zindi.africa/hackathons/umojahack-rwanda-expresso-churn-prediction-challenge/data), we are provided with everything we need to get started:

Train.csv: this is what you will use to train your model.

Test.csv: this is what you will test your model on.

SampleSubmission.csv: This file serves as an example for how to format your submission.

Let's start by importing the libraries that we will need to load and explore the data.

In [3]:
##importing packages
import pandas as pd 
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
# import missingno as msno

Next, we can load the datasets and begin taking a look.

In [ ]:
#reading in the data
train = pd.read_csv('UmojaHack/Train.csv')
test = pd.read_csv('UmojaHack/Test.csv')
submission_file = pd.read_csv('UmojaHack/SampleSubmission.csv')
describtion = pd.read_csv('UmojaHack/VariableDefinitions.csv') 

In [ ]:
train.head()

In [ ]:
train.head(20)

Each row is/was an expresso client. The columns are the different attributes of the customers. 

In [ ]:
train.info()#quite the data

There are more than 2M customers in the training set.

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data[missing_data.Total > 0]

In [ ]:
total = test.isnull().sum().sort_values(ascending=False)
percent = (test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data[missing_data.Total > 0]

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train.describe(exclude = np.number)#no duplicates in user_id, MRG is useless

In [ ]:
test.describe(exclude = np.number)

Let's delete the 'MRG' column since it consists of a single value. Also, let's delete colums 'ZONE2' and 'ZONE1' since they have so many missing values. 

In [ ]:
train.REGION.unique()

In [ ]:
print(train.shape)
print()
print(test.shape)

# EXPLORING THE DATA

Let's start by looking at the distribution of our target variable.

In [ ]:
print(train.groupby(['CHURN']).size())
#bar chart to show distribution of the target variable
n_obs = train.shape[0]
#index = ['No','Yes']
churn_plot = train['CHURN'].value_counts().div(n_obs).plot(kind='bar',figsize=(4,4),title="Churn rate", color=['#BB6B5A','#8CCB9B'])
churn_plot.set_xlabel("Churn Risk")
churn_plot.set_ylabel("Frequency")

It is imbalanced, with more than 80% of customers staying. Next, let's take a look at our features. We have a mix of continous and categorical features. 

Let's begin with the categorical features and investigate how the churn rate differ across the various levels of each categorical feature.

In [ ]:
def attrition_rate_plot(col, target, data, ax=None):
    """Stacked bar chart of churn rate for `target` against 
    `col`. 
    
    Args:
        col (string): column name of feature variable
        target (string): column name of target variable
        data (pandas DataFrame): dataframe that contains columns 
            `col` and `target`
        ax (matplotlib axes object, optional): matplotlib axes 
            object to attach plot to
    """
    counts = (train[[target, col]]
                  .groupby([target, col])
                  .size()
                  .unstack(target)
             )
    group_counts = counts.sum(axis='columns')
    props = counts.div(group_counts, axis='index')

    props.plot(kind="barh", stacked=True, ax=ax, color = ['g', 'r'])
    ax.invert_yaxis()
    ax.legend().remove()

In [ ]:
cols_to_plot = [
    'REGION',
    'TENURE'
]

fig, ax = plt.subplots(
    len(cols_to_plot), 1, figsize=(11,11)
)
for idx, col in enumerate(cols_to_plot):
    attrition_rate_plot(
        col, 'CHURN', train, ax=ax[idx]
    )

ax[0].legend(
    loc='lower center', bbox_to_anchor=(0.5, 1.05), title='Churn'
)

fig.tight_layout()

It looks like customers with tenure between 12 and 15 months have the highest churn rate while new customers have the lowest.

Now, let's look at the relationship between the recharge amount and the monthly revenue

In [ ]:
# Set initial plot options
sns.set_style('white')
plt.figure(figsize = (14, 8))

# Create scatterplot
sns.scatterplot(x = "MONTANT", 
                y = "REVENUE", 
                # Group by and change dot style and  by CHURN
                hue = "CHURN",
                size = "CHURN",  
                style = "CHURN", 
                data = train, 
                # Change color of hue categories
                palette = ["r", "g"],
                alpha = 0.2)

# Despine plot
sns.despine()
# Final formatting touches
plt.xlabel("MONTANT", fontsize = 12, fontweight = "semibold")
plt.ylabel("REVENUE", fontsize = 12, fontweight = "semibold")
plt.title("REVENUE by AMOUNT", fontsize = 14, fontweight = "semibold")
plt.show()

There seems to be some correlation between the two and also there's some outliers. Outliers can create bias in a model's performance. Should we keep them or remove them? I will let you decide.

# BUILDING SOME MODELS

Let's train a model using the [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). We chose the logistic regression because it is fast and serve as a good baseline.

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score

RANDOM_SEED = 123    # Set a random seed for reproducibility!

Some models, including logistic regression, only work with numeric input for features. So we'll either have to drop the categorical features or transform them. We'll opt for the later since we want to capture as much as information as we can. To do this, we'll use a method called [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).

We also need to do some preprocessing for the numeric features. We have to scale each numeric feature and the reason for that is because we are using [regularization](https://en.wikipedia.org/wiki/Regularization_(mathematics)). We will use the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html), it transforms each feature such that its distribution has a mean value of 0 and a standard deviation of 1.

Another issue we have to take care of is the missing values since logistic regression can't handle them. For that, we'll use theSimpleImputer (https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) function.

This is quite a lot of steps but not to worry, we'll use Scikit-Learn's built-in composition functionality to encapsulate everything into a pipeline. 

### Feature Preprocessing

In [16]:
#reading in the data
train = pd.read_csv('UmojaHack/Train.csv')
test = pd.read_csv('UmojaHack/Test.csv')
submission_file = pd.read_csv('UmojaHack/SampleSubmission.csv')
describtion = pd.read_csv('UmojaHack/VariableDefinitions.csv') 


In [ ]:
# import re
# preprocess = pd.DataFrame()

# #preprocess['TOP_PACK'] = train.TOP_PACK

# to_extracts = {
#    'p_unlimited':r'unlimited|ILLIMITE',
#    'p_onnet':r'On ?net',
#    'p_offnet':r'Off',
#    'p_pilot':r'pilot',
#    'p_jokko':r'JOKKO',
#    'p_social':r'FACEBOOK|TWTER',
#    'p_week':r'WEEK|7',
#    'p_month':r'MONTH|30',
# }
# for name, exp in to_extracts.items():
#     filt = train.TOP_PACK.str.contains(exp, flags=re.IGNORECASE, na=False)
#     preprocess[name] = filt * 1

# preprocess


In [6]:
# train = train.drop(['MRG', 'ZONE2', 'ZONE1'], axis = 1)
# test = test.drop(['MRG', 'ZONE2', 'ZONE1'], axis = 1)
# train['ZONE'] = train[['ZONE1','ZONE2']].sum(axis=1)
# test['ZONE'] = test[['ZONE1','ZONE2']].sum(axis=1)

train = train.drop(['user_id', 'MRG', 'ZONE2', 'ZONE1'], axis = 1)
test = test.drop(['user_id', 'MRG', 'ZONE2', 'ZONE1'], axis = 1)

# def prep(train):

feat_operator = train[['ON_NET', 'ORANGE', 'TIGO']]
feat_operator = feat_operator.apply(lambda x:x.fillna(x.value_counts().index[0]))
train[['ON_NET', 'ORANGE', 'TIGO']] = feat_operator
#------------------------------------------
columns_means = ['MONTANT', 'FREQUENCE_RECH', 'FREQUENCE', 'REVENUE', 'FREQ_TOP_PACK']

for column_mean in columns_means:
    mean_rev = round(np.log(train[[column_mean]].mean()[0]), 2)
    train[column_mean] = np.log(train[[column_mean]].fillna(mean_rev))
#-------------------------------------
mean_rev = round(np.log(1+train[['ARPU_SEGMENT']].mean()[0]), 2)
train['ARPU_SEGMENT'] = np.log(1+train[['ARPU_SEGMENT']].fillna(mean_rev))

train['REGION'] = train[['REGION']].fillna('unknown')

#     fill_list = list(train[['DATA_VOLUME']].dropna().values.reshape(-1))

#     m = train['DATA_VOLUME'].isnull()
#     #set NaNs values
#     train.loc[m, 'DATA_VOLUME'] = np.random.choice(fill_list, size=m.sum())
#     train['DATA_VOLUME'] = np.log(1+train[['DATA_VOLUME']])
#     return train

# train = prep(train)
# test = prep(test)

In [ ]:
train.columns

In [ ]:
# train.columns[train.dtypes != "object"].values

In [7]:
numeric_cols = ['MONTANT', 'FREQUENCE_RECH', 'REVENUE', 'ARPU_SEGMENT',
       'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO',
       'REGULARITY', 'FREQ_TOP_PACK']
print(numeric_cols)

['MONTANT', 'FREQUENCE_RECH', 'REVENUE', 'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO', 'REGULARITY', 'FREQ_TOP_PACK']


In [8]:
#categorical_cols = df.columns[df.dtypes == "object"].values
categorical_cols = ['REGION', 'TENURE', 'TOP_PACK']
print(categorical_cols)

['REGION', 'TENURE', 'TOP_PACK']


In [9]:
# chain preprocessing into a Pipeline object
# each step is a tuple of (name you chose, sklearn transformer)
numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='constant', fill_value=0, add_indicator = True))
])

categorical_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='constant', fill_value='Missing', add_indicator = True)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# create the preprocessor stage of final pipeline
# each entry in the transformer list is a tuple of
# (name you choose, sklearn transformer, list of columns)
preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols),
        ('categorical', categorical_preprocessing_steps, categorical_cols)
    ],
    remainder = "drop"
)

### Putting Together the Full Pipeline

We put both the preprocessing functions and the estimatior into one Pipeline object, this allows to run the data through all the steps in one interface.

In [10]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
full_pipeline_LGBM = Pipeline([
    ("preprocessor", preprocessor),
    ("estimator", LGBMClassifier()
),
])

full_pipeline_XGB = Pipeline([
    ("preprocessor", preprocessor),
    ("estimator", XGBClassifier(max_depth=20)
),
])

In [15]:
train.reset

,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,FATICK,K > 24 month,4250.000000,15.00000,4251.000000,1417.000000,17.000000,4.0,388.0,46.0,1.0,54,On net 200F=Unlimited _call24H,8.000000,0
1,unknown,I 18-21 month,5532.116998,11.52912,5510.810334,1836.942894,13.978141,NaN,0.0,1.0,1.0,4,NaN,9.272461,1
2,unknown,K > 24 month,3600.000000,2.00000,1020.000000,340.000000,2.000000,NaN,90.0,46.0,7.0,17,On-net 1000F=10MilF;10d,1.000000,0
3,DAKAR,K > 24 month,13500.000000,15.00000,13502.000000,4501.000000,18.000000,43804.0,41.0,102.0,2.0,62,"Data:1000F=5GB,7d",11.000000,0
4,DAKAR,K > 24 month,1000.000000,1.00000,985.000000,328.000000,1.000000,NaN,39.0,24.0,1.0,11,Mixt 250F=Unlimited_call24H,2.000000,0


Let's check out the full pipeline.

In [ ]:
# from sklearn import set_config

# full_pipeline

### Training and Evaluation

Let's split the data into a training and evaluation set. We'll use a third of our data for evaluation. 

Recall that we have an imbalanced dataset, so we'll use the stratify argument to enforce even splits.

In [11]:
X_train, X_eval, y_train, y_eval = train_test_split(
    train.drop(['CHURN'], axis = 1),
    train.CHURN,
    test_size=0.20,
    shuffle=True,
    stratify=train.CHURN,
    random_state=RANDOM_SEED
)

In [13]:
%%time

# Train model
full_pipeline_XGB.fit(X_train, y_train)

# Predict on evaluation set
# This competition wants probabilities, not labels
preds_XGB = full_pipeline_XGB.predict_proba(X_eval)
preds_XGB

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_eval, preds_XGB[:, 1])

In [14]:
%%time

# Train model
full_pipeline_LGBM.fit(X_train, y_train)

# Predict on evaluation set
# This competition wants probabilities, not labels
preds_LGBM = full_pipeline_LGBM.predict_proba(X_eval)
preds_LGBM

CPU times: user 28.6 s, sys: 292 ms, total: 28.9 s
Wall time: 28.9 s


array([[0.99580003, 0.00419997],
       [0.94133634, 0.05866366],
       [0.79959716, 0.20040284],
       ...,
       [0.99408769, 0.00591231],
       [0.99822307, 0.00177693],
       [0.92976585, 0.07023415]])

In [ ]:
log_loss(y_eval, preds_LGBM[:, 1])

In [ ]:
new_preds = 1/2*(preds_LGBM[:, 1]+preds_LGBM[:, 1])

In [ ]:
log_loss(y_eval, new_preds)

In [ ]:
from sklearn.metrics import classification_report
# print(classification_report(np.array(y_eval), preds[:, 1])) 

The competition uses the [Log loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html) as the evaluation metric, so let's check how our model did on the evaluation set.

The estimator spits out the probabilities for each class(0 and 1). We are interested in the second column, that is the probability of churn,so let's grab it.

In [ ]:
# import scipy as sp
# import scikitplot as skplt

In [ ]:
# print('Confusion matrix:\n', skplt.metrics.plot_confusion_matrix(y_eval, preds[:, 1].round()))

### Retrain Model on Full Dataset

Now that we have an idea of how the model performs, let's retrain on the full training set

In [ ]:
# %%time 

# full_pipeline.fit(train.drop(['user_id', 'CHURN'], axis = 1), train.CHURN)

# None   # So we don't print out the whole pipeline representation

# GENERATING THE PREDICTIONS FOR THE TEST SET

Let's make predictions on the test set! Remember, for this competition, we want the probabilities, not the binary label predictions. So just like we did earlier, we'll use the .predict_proba method to get those.

In [ ]:

# feat_operator = test[['ON_NET', 'ORANGE', 'TIGO']]
# feat_operator = feat_operator.apply(lambda x:x.fillna(x.value_counts().index[0]))
# test[['ON_NET', 'ORANGE', 'TIGO']] = feat_operator
# #------------------------------------------
# columns_means = ['MONTANT', 'FREQUENCE_RECH', 'FREQUENCE', 'REVENUE', 'FREQ_TOP_PACK']

# for column_mean in columns_means:
#     mean_rev = round(np.log(test[[column_mean]].mean()[0]), 2)
#     test[column_mean] = np.log(test[[column_mean]].fillna(mean_rev))
# #-------------------------------------
# mean_rev = round(np.log(1+test[['ARPU_SEGMENT']].mean()[0]), 2)
# test['ARPU_SEGMENT'] = np.log(1+test[['ARPU_SEGMENT']].fillna(mean_rev))

# test['REGION'] = test[['REGION']].fillna('unknown')


In [ ]:
test_probas_xgb = full_pipeline_XGB.predict_proba(test)
test_probas_xgb

In [ ]:
test_probas_LGBM = full_pipeline_LGBM.predict_proba(test)
test_probas_LGBM

In [ ]:
new_preds = 1/2*(test_probas_xgb[:, 1]+test_probas_LGBM[:, 1])

Let's use the submission file to submit the predicted values.

In [ ]:
submission_df = pd.read_csv('UmojaHack/SampleSubmission.csv')
submission_df.head()

We want to replace those 0s with our predictions. But first, we need to make sure that the rows of the submission file are in the same order as the test file. 

In [ ]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test.index.values, 
                              submission_df.index.values)

Nothing happended, so we're good. We can safely can drop in the estimated values in the 'CHURN' column.

In [ ]:
# Save predictions to submission data frame
submission_df["CHURN"] = new_preds #test_probas_xgb[:, 1]

submission_df.head()

In [ ]:
submission_df.to_csv('my_submission_4.csv', index=False)

In [ ]:
!head my_submission.csv

Now, let's on head on over to [Zindi](https://zindi.africa/hackathons/umojahack-rwanda-expresso-churn-prediction-challenge) and make our submission.

# Things to try

* [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find optimal hyper parameters. If you do chose to use gridsearch, I suggest splitting the dataset into small portions since the it is so large
* Mean target encoding
* [Principal component analysis](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) (PCA)
* Removing outliers
* Try different algorithms